# Exercise 11 : Gluon Path Integrals

### Function to generate an array of SU(3) matrices and their inverses to use in the monte-carlo process

See Gattringer and Lang page 83. Although they say you should choose random numbers between $(-0.5, 0.5)$ this leads to negative numbers on the diagonal for the generated su3 matrices, these are not closer to the identity for smaller epsilon (as -.999 is generated for epsilon = 0.01 and -.075 is generated for epsilon = 0.5, the first value is obviously further away from 1 even though the epsilon is smaller). Hence smaller epsilon does not give a higher acceptance which is required to tune it for 50%

So I will use random numbers from $(0,0.5)$ I tested the expectation values obtained from $(0,0.5)$ and $(-0.5,0.5)$ and they are the same. After emailing lang, only need 3 random numbers and r[0] = sqrt(xxx) without sign function

Need to examine error propagation, autocorrelation

In [1]:
import numpy as np
import os
from re import split
from multiprocessing import Pool

def generate_su2(epsilon):
    #generates a random su2 matrix where epsilon controls the "distance" from the identity
    r_random_numbers = np.random.uniform(0, 0.5, (4))
    r = np.empty((4))
    
    r[1:] = epsilon * r_random_numbers[1:] / np.linalg.norm(r_random_numbers[1:])
    r[0] = np.sign(r_random_numbers[0]) * np.sqrt(1-epsilon**2)
    
    r11 = r[0] + r[3]*1j
    r12 = r[1]*1j + r[2]
    r21 = r[1]*1j - r[2]
    r22 = r[0] - r[3]*1j
    
    return np.array([[r11, r12], [r21, r22]])
    
def generate_su3_array(n, epsilon):
    #generates a 2*n array of su3 matrices where epsilon controls the distance from the idenity
    su3_array = np.empty((2*n, 3, 3), dtype = 'complex128')
    
    for i in range(n):

        R_su3 = np.identity(3, dtype=complex)
        S_su3 = np.identity(3, dtype=complex)
        T_su3 = np.identity(3, dtype=complex)

        R_su3[:2, :2] = generate_su2(epsilon)
        S_su3[0:3:2, 0:3:2] = generate_su2(epsilon)
        T_su3[1:, 1:] = generate_su2(epsilon)
        
        X_su3 = np.dot(np.dot(R_su3, S_su3), T_su3)

        su3_array[2*i, :, :] = X_su3
        su3_array[2*i+1, :, :] = X_su3.conj().T
        
    return su3_array

### Classes for the lattice and the site at each lattice point.

Site value is just a scalar here as this is a simple gauge simulation. The fundamental data structure of the lattice is a numpy array of site objects which have four links initialised to the identity matrix.

In [2]:
def generate_site():
    #Returns a 4 dimensional array of su3 matrices initialised to the identity which can be assigned to each
    #spatial lattice point
    link_value = np.identity(3, dtype = 'complex128')
    link = np.tile(link_value, (4, 1, 1))
    
    return link
        

def generate_lattice(n_points):
    #Returns a (n_points, n_points, n_points, n_points, 4, 3, 3) numpy array as our lattice with 4
    #su3 links assigned to each grid point
    volume = np.append(np.repeat(n_points, 4), (4,3,3))
    grid = np.empty(volume, dtype = 'complex128')
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    grid[t, x, y, z, :, :, :] = generate_site()
                        
    return grid

def link(lattice, coordinates, mu):
    #Function to take into account periodic bd conditions
    n_points = lattice.shape[0]
        
    return lattice[coordinates[0] % n_points, coordinates[1] % n_points, 
                   coordinates[2] % n_points, coordinates[3] % n_points, mu, : , :]

### Function which computes the staple sum for a given site link

The function takes a coordinate array [t, x, y, z] for the lattice and a direction t = 0, x = 1, y = 2 or z = 3 and computes the sum of staples which include this link. This corresponds to the Wilson action. See Gattringer and Lang page 79 eqn (4.20).

In [3]:
def wilson_link_sum(lattice, coordinates,  mu, u_0):
    #Staple sum for the wilson plaquette action
    dimension = 4
    res = np.zeros((3,3), dtype = 'complex128')
    
    for nu in range(dimension):
        if nu != mu:
            
            coordinates_mu = coordinates[:]
            coordinates_mu[mu] += 1
            
            coordinates_nu = coordinates[:]
            coordinates_nu[nu] += 1
            
            coordinates_mu_n_nu = coordinates[:]
            coordinates_mu_n_nu[mu] += 1
            coordinates_mu_n_nu[nu] -= 1
            
            coordinates_n_nu = coordinates[:]
            coordinates_n_nu[nu] -= 1
            
            #1x1 positive
            res += np.dot(np.dot(link(lattice, coordinates_mu, nu), 
                                 link(lattice, coordinates_nu, mu).conj().T),
                                 link(lattice, coordinates,    nu).conj().T)
            #1x1 negative
            res += np.dot(np.dot(link(lattice, coordinates_mu_n_nu, nu).conj().T, 
                                 link(lattice, coordinates_n_nu,    mu).conj().T), 
                                 link(lattice, coordinates_n_nu,    nu))
        
    return res / u_0**4 / 3

def improved_link_sum(lattice, coordinates, mu, u_0):
    #Staple sum for rectangular improved action which includes next nearest neighbour links
    
    dimension = 4
    res = np.zeros((3,3), dtype = 'complex128')
    res_rec = np.zeros((3,3), dtype = 'complex128')
    
    for nu in range(dimension):
        if nu != mu:
            
            coordinates_mu = coordinates[:]
            coordinates_mu[mu] +=1
            
            coordinates_nu = coordinates[:]
            coordinates_nu[nu] +=1
            
            coordinates_mu_nu = coordinates[:]
            coordinates_mu_nu[mu] +=1
            coordinates_mu_nu[nu] +=1
            
            coordinates_mu_mu = coordinates[:]
            coordinates_mu_mu[mu] += 2
            
            coordinates_nu_nu = coordinates[:]
            coordinates_nu_nu[nu] += 2
            
            coordinates_mu_n_nu = coordinates[:]
            coordinates_mu_n_nu[mu] += 1
            coordinates_mu_n_nu[nu] -= 1
            
            coordinates_n_nu = coordinates[:]
            coordinates_n_nu[nu] -= 1
            
            coordinates_n_nu_nu = coordinates[:]
            coordinates_n_nu_nu[nu] -= 2
            
            coordinates_mu_mu_n_nu = coordinates[:]
            coordinates_mu_mu_n_nu[mu] += 2
            coordinates_mu_mu_n_nu[nu] -= 1
            
            coordinates_mu_n_nu_nu = coordinates[:]
            coordinates_mu_n_nu_nu[mu] += 1
            coordinates_mu_n_nu_nu[nu] -= 2
            
            coordinates_n_mu_nu = coordinates[:]
            coordinates_n_mu_nu[mu] -= 1
            coordinates_n_mu_nu[nu] += 1
            
            coordinates_n_mu = coordinates[:]
            coordinates_n_mu[mu] -= 1
            
            coordinates_n_mu_n_nu = coordinates[:]
            coordinates_n_mu_n_nu[mu] -= 1
            coordinates_n_mu_n_nu[nu] -= 1
    
            link_00_nu = link(lattice, coordinates,       nu)
            link_01_mu = link(lattice, coordinates_nu,    mu)
            link_01_nu = link(lattice, coordinates_nu,    nu)
            link_10_mu = link(lattice, coordinates_mu,    mu)
            link_10_nu = link(lattice, coordinates_mu,    nu)
            link_11_mu = link(lattice, coordinates_mu_nu, mu)
            link_11_nu = link(lattice, coordinates_mu_nu, nu)
            link_20_nu = link(lattice, coordinates_mu_mu, nu)
            link_02_mu = link(lattice, coordinates_nu_nu, mu) 
            
            link_0n1_mu = link(lattice, coordinates_n_nu,       mu)
            link_0n1_nu = link(lattice, coordinates_n_nu,       nu)
            link_0n2_mu = link(lattice, coordinates_n_nu_nu,    mu)
            link_0n2_nu = link(lattice, coordinates_n_nu_nu,    nu)
            link_1n1_mu = link(lattice, coordinates_mu_n_nu,    mu)
            link_1n1_nu = link(lattice, coordinates_mu_n_nu,    nu)
            link_1n2_nu = link(lattice, coordinates_mu_n_nu_nu, nu)
            link_2n1_nu = link(lattice, coordinates_mu_mu_n_nu, nu)
            
            link_n10_nu =  link(lattice, coordinates_n_mu,      nu)
            link_n10_mu =  link(lattice, coordinates_n_mu,      mu)
            link_n11_mu =  link(lattice, coordinates_n_mu_nu,   mu)
            link_n1n1_mu = link(lattice, coordinates_n_mu_n_nu, mu)
            link_n1n1_nu = link(lattice, coordinates_n_mu_n_nu, nu)
            
            #1x1 above
            res += np.dot(np.dot(link_10_nu, 
                                 link_01_mu.conj().T),
                                 link_00_nu.conj().T)
            #1x1 below
            res += np.dot(np.dot(link_1n1_nu.conj().T, 
                                 link_0n1_mu.conj().T), 
                                 link_0n1_nu)
            
            #2x1 landscape rectangle in front and above
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_mu,
                                                   link_20_nu),
                                                   link_11_mu.conj().T), 
                                                   link_01_mu.conj().T),
                                                   link_00_nu.conj().T)
            
            #2x1 landscape rectangle in front and below
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_mu,
                                                   link_2n1_nu.conj().T),
                                                   link_1n1_mu.conj().T), 
                                                   link_0n1_mu.conj().T),
                                                   link_0n1_nu)
            
            #2x1 landscape rectangle behind and above
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_nu,
                                                   link_01_mu.conj().T),
                                                   link_n11_mu.conj().T), 
                                                   link_n10_nu.conj().T),
                                                   link_n10_mu)
            
            #2x1 landscape rectangle behind and below
            res_rec += np.dot(np.dot(np.dot(np.dot(link_1n1_nu.conj().T,
                                                   link_0n1_mu.conj().T),
                                                   link_n1n1_mu.conj().T), 
                                                   link_n1n1_nu),
                                                   link_n10_mu)
            
            #1x2 portrait rectangle above
            res_rec += np.dot(np.dot(np.dot(np.dot(link_10_nu,
                                                   link_11_nu),
                                                   link_02_mu.conj().T), 
                                                   link_01_nu.conj().T),
                                                   link_00_nu.conj().T)
            
            #1x2 portrait rectangle below
            res_rec += np.dot(np.dot(np.dot(np.dot(link_1n1_nu.conj().T,
                                                   link_1n2_nu.conj().T),
                                                   link_0n2_mu.conj().T), 
                                                   link_0n2_nu),
                                                   link_0n1_nu)
    
    return (5 * res / u_0**4 / 9) - (res_rec / u_0**6 / 36)

### Update function for a single gauge link

The function takes a su3 matrix, coordinate array, direction and constant beta and preforms a single montecarlo step for a single gauge link

In [4]:
def update_link(lattice, coordinates, mu, link_sum, su3_set, su3_set_length, beta, u_0, n_hits):
    
    accept = 0
    
    t = coordinates[0]
    x = coordinates[1]
    y = coordinates[2]
    z = coordinates[3]
    
    #compute staple once for given link
    staple = link_sum(lattice, coordinates,  mu, u_0)
    
    #update single link multiple times to bring into equilibrium with surrounding links
    for i in range(n_hits):

        su3_matrix = su3_set[np.random.randint(0, su3_set_length)]
        
        new_link = np.dot(su3_matrix , lattice[t, x, y, z, mu, : , :])
        
        diff = new_link - lattice[t, x, y, z, mu, : , :]
    
        deltaS = beta * np.trace(np.dot(diff, staple)).real
    
        if np.exp(deltaS) > np.random.rand():
          
            lattice[t, x, y, z, mu, : , :] = new_link
            accept += 1
    
    return accept / n_hits

### Function to update whole lattice

In [5]:
def update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits):
    n_points = lattice.shape[0]
    dimension = 4 
    acceptance = 0
    su3_set_length = len(su3_set)
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        acceptance += update_link(lattice, [t,x,y,z], mu, link_sum, 
                                                  su3_set, su3_set_length, beta, u_0, n_hits)
                            
    return acceptance / dimension / n_points**dimension

### Ensemble Generation

Do individual hits count as updates for correlation? assumming they do and 50 correlation steps = 5 correlation x 10 hits

In [6]:
def markov_chain(lattice, link_sum,  su3_set, beta, u_0, n_configs, n_corr, n_hits = 10, 
                 thermal_steps = 100, thermalise = True, save = True):
    
    acceptance = 0
    
    #thermalise
    if thermalise == True:
        for i in range(thermal_steps):
            
            update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits)
    
    #equilibrium update
    for j in range((n_configs-1)*n_corr):
        
        acceptance += update_lattice(lattice, link_sum, su3_set, beta, u_0, n_hits)
        
        if save == True:
            if j % n_corr == 0:
            
                ensemble_index = int(j/n_corr)
                config_save(lattice, link_sum.__name__, beta, ensemble_index)
    
    if n_configs != 0 and n_corr != 0:
    
        acceptance = acceptance / n_configs / n_corr       
    
    return acceptance

### Plaquette lattice average

In [7]:
def square_wilson_loop(lattice, coordinates, mu, nu):
    #compute upper plaquette
    
    coordinates_mu = coordinates[:]
    coordinates_mu[mu] +=1
    coordinates_nu = coordinates[:]
    coordinates_nu[nu] +=1
    
    res = np.dot(np.dot(np.dot(link(lattice, coordinates,    mu), 
                               link(lattice, coordinates_mu, nu)), 
                               link(lattice, coordinates_nu, mu).conj().T), 
                               link(lattice, coordinates,    nu).conj().T)
        
    return np.trace(res).real / 3


def square_lattice_average(lattice):
    #average plaquette over whole lattice
    dimension = 4
    unique_2d_loops_number = 6
    n_points = lattice.shape[0]
    
    res = 0
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        for nu in range(mu):
                            res += square_wilson_loop(lattice, [t,x,y,z], mu, nu)
                                
    return res / n_points**dimension / unique_2d_loops_number

### Rectangle loop lattice average

In [8]:
def rectangular_wilson_loop(lattice, coordinates, mu, nu):
    #compute upper rectangle in front
    
    coordinates_mu = coordinates[:]
    coordinates_mu[mu] += 1
    coordinates_nu = coordinates[:]
    coordinates_nu[nu] += 1
    coordinates_mu_nu = coordinates[:]
    coordinates_mu_nu[mu] += 1
    coordinates_mu_nu[nu] += 1
    coordinates_mu_mu = coordinates[:]
    coordinates_mu_mu[mu] += 2        
    
    link_00_mu = link(lattice, coordinates,       mu)
    link_00_nu = link(lattice, coordinates,       nu)
    link_10_mu = link(lattice, coordinates_mu,    mu)
    link_11_mu = link(lattice, coordinates_mu_nu, mu)
    link_01_mu = link(lattice, coordinates_nu,    mu)
    link_20_nu = link(lattice, coordinates_mu_mu, nu) 
    
    #2x1 positive rectangle
    res = np.dot(np.dot(np.dot(np.dot(np.dot(link_00_mu, 
                                             link_10_mu),
                                             link_20_nu),
                                             link_11_mu.conj().T), 
                                             link_01_mu.conj().T),
                                             link_00_nu.conj().T)
    
    return np.trace(res).real / 3


def rectangular_lattice_average(lattice):
    #average landscape & portrait rectangle over lattice
    dimension = 4
    unique_2d_loops_number = 6
    n_points = lattice.shape[0]
    
    res = 0
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        for nu in range(mu):
                            res += rectangular_wilson_loop(lattice, [t,x,y,z], mu, nu)
                            res += rectangular_wilson_loop(lattice, [t,x,y,z], nu, mu)
                                
    return res / n_points**dimension / unique_2d_loops_number / 2

### Config save and load

In [9]:
def config_save(configuration, action, beta, k):
    #save in a local folder "gauge_configs"
    filename = "gauge_configs/" + str(action) + "/beta_" + str(beta) + "/" + str(k)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    np.save(filename, configuration, allow_pickle = True)
    print("Configuration " + str(k) + " saved")

def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in split(r'(\d+)', string_)]

def load_ensemble(action, beta):
    #load ensemble files using action = "wilson_link_sum" or "improved_link_sum"
    ensemble = []
    directory = "gauge_configs/" + str(action) + "/beta_" + str(beta)  + "/"
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in sorted(files, key = natural_key):
            ensemble.append(np.load(os.path.abspath(os.path.join(root, name)), allow_pickle=True))
                        
    return ensemble

def gauge_average(lattice_ensemble, lattice_average):
    
    p = Pool(6)
    loop_ensemble = list(p.map(lattice_average, lattice_ensemble))
    p.terminate()
    
    return np.mean(loop_ensemble), np.std(loop_ensemble) / np.sqrt(len(lattice_ensemble))

### Analysis

In [10]:
latt1 = generate_lattice(n_points = 8)

su3_array1 = generate_su3_array(25, epsilon = 0.2)

acceptance1 = markov_chain(latt1, wilson_link_sum, su3_array1, beta = 5.5, u_0= 1, 
                           n_configs = 200, n_corr = 50, n_hits = 10)

latt2 = generate_lattice(n_points = 8)

su3_array2 = generate_su3_array(25, epsilon = 0.2)

acceptance2 = markov_chain(latt2, improved_link_sum, su3_array2, beta = 1.719, u_0= 0.797, 
                           n_configs = 200, n_corr = 50, n_hits = 10)

Configuration 0 saved
Configuration 1 saved
Configuration 2 saved
Configuration 3 saved
Configuration 4 saved
Configuration 5 saved
Configuration 6 saved
Configuration 7 saved
Configuration 8 saved
Configuration 9 saved
Configuration 10 saved
Configuration 11 saved
Configuration 12 saved
Configuration 13 saved
Configuration 14 saved
Configuration 15 saved
Configuration 16 saved
Configuration 17 saved
Configuration 18 saved
Configuration 19 saved
Configuration 20 saved
Configuration 21 saved
Configuration 22 saved
Configuration 23 saved
Configuration 24 saved
Configuration 25 saved
Configuration 26 saved
Configuration 27 saved
Configuration 28 saved
Configuration 29 saved
Configuration 30 saved
Configuration 31 saved
Configuration 32 saved
Configuration 33 saved
Configuration 34 saved
Configuration 35 saved
Configuration 36 saved
Configuration 37 saved
Configuration 38 saved
Configuration 39 saved
Configuration 40 saved
Configuration 41 saved
Configuration 42 saved
Configuration 43 save

In [11]:
ensemble1 = load_ensemble("wilson_link_sum", beta = 5.5)

rect_plaquette = gauge_average(ensemble1, square_lattice_average)

rect_rect = gauge_average(ensemble1, rectangular_lattice_average)

print("Wilson action square loop expectation " + str(rect_plaquette) )
print("Wilson action rectangular loop expectation " + str(rect_rect))


Wilson action square loop expectation (0.49693624646771889, 0.00033897665809338914)
Wilson action rectangular loop expectation (0.25987224762809213, 0.00043437882625035055)


In [12]:
ensemble2 = load_ensemble("improved_link_sum", beta = 1.719)

rect_plaquette = gauge_average(ensemble2, square_lattice_average)

rect_rect = gauge_average(ensemble2, rectangular_lattice_average)

print("Rectangular action square loop expectation " + str(rect_plaquette) )
print("Rectangular action rectangular loop expectation " + str(rect_rect))


Rectangular action square loop expectation (0.54132682334711146, 0.00022397773139215416)
Rectangular action rectangular loop expectation (0.28383735574200769, 0.00031803901503908217)
